합성곱 신경망
CNN은 이미지 인식과 음성 인식 등 다양한 곳에서 사용된다. 특히 이미지 인식분야에서 딥러닝을 활용한 기법은 거의 다 CNN을 기초로 한다.
CNN의 네트워크 구조
합성곱 계층
풀링 계층
지금 까지 본 신경망은 인접하는 계층의 모든 뉴런과 결합되어 있다. 이를 완전연결이라고 하며완전히 연결된 계층을 Affine계층이라 부른다.
Affine계층뒤에 활성화 함수를 갖는다.
CNN은 Conv-ReLU-(Pooling)흐름으로 연결된다. 
CNN에서는 패딩, 스트라이드라는 용어가 등장한다. 각 계층 사이에는 3차원 데이터와 같이 입체적인 데이터가 흐른다는 점에서 완전연결 신경망과 다르다.
완전연결 계층의 문제점
완전연결 계층은 인접하는 계층의 뉴런이 모두 연결되고 출력의 수는 임의로 정할 수 있습니다.
이러한 문제점은 데이터의 형상이 무시된다는 것입니다.
이미지의 경우 통상 세로 가로 채널로 구성된 3차원 데이터입니다. 그러나 완전연결 계층에 입력할 때는 3차원 데이터를 평평한 1차원 데이터로 평탄화해줘야합니다.
이미지는 3차원 형상이기때문에 공간정보가 담겨있다.RGB나 픽셀등 본질적인 패턴이 의미가 있는것이 숨겨져있을 수 있다. 이러한 정보를 무시함

합성공 계층은 이러한 형상을 유지할 수 있다.
CNN에서는 합성곱계층의 입출력 데이터를 특징 맵이라고 한다.
입력데이터를 입력 특징 맵, 출력 데이터를 출력 특징 맵이라한다.
합성곱 연산
이미지 처리에서 말하는 필터 연산에 해당한다.
합성곱연산에는 입력 데이터와 필터가 적용되는데 이 필터를 커널이라고 한다.
합성곱 연산은 필터의 윈도우를 일정 간격으로 이동해가며 입력데이터에 적용합니다.
여기서 윈도우는 3*3부분을 가리킨다.
그 후 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합을 구합니다.(단일 곱셈-누산) 그 결과를 출력의 해당장소에 저장합니다.
CNN에서 필터의 매개변수가 가중치에 해당한다. CNN에도 편향이 존재한다.
출력값에+편향을 하는 것이다.

패딩
합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값으로 채우기도 한다. 이를 패딩이라 한다.

스트라이드
필터를 적용하는 위치의 간격
스트라이드가 2라면 3*3을 한 후 2칸 옆으로 이동
OH=(H+2P-FH)/S+1
OW=(W+2P-FW)/S+1
결과값이 나누어 떨어져야함

3차원 데이터의 합성곱 연산
입력데이터의 채널수와 필터의 채널 수가 같아야한다.

필터의 크기는 상관없음



7.3 풀링 계층
세로 가로 방향의 공간을 줄이는 연산이다.
2*2 최대풀링을 스트라이드 2로 처리하는 순서이다.
최대 풀링은 최댓값을 구하는 연산으로 2*2는 대상영역의 크기를 뜻한다. 
만약 윈도우 4*4면 스트라이드는 4로 설정한다.
풀링 계층의 특징
1. 학습해야할 매개변수가 없다.
2. 채널수가 변하지 않는다.//풀링 연산은 입력 데이터의 채널 수 그대로 출력 데이터로 내보낸다.
3.입력의 변화에 영향을 적게 받는다.


4차원 배열
CNN에서 계층 사이를 흐르는 데이터는 4차원이다. 데이터의 형상이 (10,1,28,28)이면 높이 28 너비28 채널 1개 데이터가 10개이다.

im2col데이터 전개
입력데이터를 필터링 하기 좋게 전개하는 함수이다. 
다차원을 2차원으로 바꿔줌
스트라이드를 크게잡아 적용영역이 겹치지 않도록 하기도 하지만 겹치면 im2col로 전개한 후 원소의 수가 원래 블록의 원소 수보다 많아진다. 그래서 im2col을 사용해 구현하면 메모리를 더 많이 소비하는 단점이 있다.
im2col로 입력데이터를 전개한 다음 합성곱 계층의 필터를 1열로 전개하고 두 행렬의 내적을 계산합니다.



In [ ]:
import numpy as np

x = np.random.rand(10, 1, 28, 28)
x.shape

In [ ]:
# coding: utf-8
import numpy as np


def smooth_curve(x):
    """손실 함수의 그래프를 매끄럽게 하기 위해 사용
    
    참고：http://glowingpython.blogspot.jp/2012/02/convolution-with-numpy.html
    """
    window_len = 11
    s = np.r_[x[window_len-1:0:-1], x, x[-1:-window_len:-1]]
    w = np.kaiser(window_len, 2)
    y = np.convolve(w/w.sum(), s, mode='valid')
    return y[5:len(y)-5]


def shuffle_dataset(x, t):
    """데이터셋을 뒤섞는다.

    Parameters
    ----------
    x : 훈련 데이터
    t : 정답 레이블
    
    Returns
    -------
    x, t : 뒤섞은 훈련 데이터와 정답 레이블
    """
    permutation = np.random.permutation(x.shape[0])
    x = x[permutation,:] if x.ndim == 2 else x[permutation,:,:,:]
    t = t[permutation]

    return x, t

def conv_output_size(input_size, filter_size, stride=1, pad=0):
    return (input_size + 2*pad - filter_size) / stride + 1


def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    
    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]


In [ ]:
import sys, os
sys.path.append(os.pardir)

x1=np.random.rand(1,3,7,7)
col1=im2col(x1,5,5,stride=1,pad=0)
print(col1.shape)

x2=np.random.rand(10,3,7,7)
col2=im2col(x2,5,5,stride=1,pad=0)
print(col2.shape)

In [ ]:
class Convolution:
    def __init__(self,W,b,stride=1,pad=0):
        self.W=W;
        self.b=b;
        self.stride=stride
        self.pad=pad
        
    def forward(self,x):
        FN,C,FH,FW=self.W.shape
        N,C,W,H=x.shape
        out_h=int(1+(H+2*self.pad-FH)/self.stride)
        out_w=int(1+(W+2*self.pad=FW)/self.stride)
        
        col=im2col(x,FH,FW,self.stride,self.pad)
        col_W=self.W.reshape(FN,-1).T
        out=np.dot(col,col_W)+self.b
        
        out=out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2)
        
        return out

transpose함수-> 출력데이터를 적절한 형상으로 바꿔준다.
다차원 배열의 축 순서를 바꿔준다.
풀링 계층 구현
1. 입력 데이터를 전개한다.
2. 행별 최댓값을 구한다.
3. 적절한 모양으로 성형한다.

CNN구현하기
합성곱 계층의 하이퍼파라미터는 딕셔너리 형태로 주어진다.

In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class SimpleConvNet:
    """단순한 합성곱 신경망
    
    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 입력 크기（MNIST의 경우엔 784）
    hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    output_size : 출력 크기（MNIST의 경우엔 10）
    activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
        'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
        'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """손실 함수를 구한다.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """기울기를 구한다（수치미분）.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """기울기를 구한다(오차역전파법).

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]


CNN의 원조 LeNet, AlexNet
LeNet->손글씨 인식 네트워크
합성곱계층과 풀링계층을 반복하고 마지막으로 완전연결계층을 거쳐결과출력
LeNet은 시그모이드 함수를 사용함
AlexNet은 활성화함수 ReLu를 사용하고LRN이라는 국소적정규화를 실시하는 계층을 이용하고 드롭아웃을 사용한다.
